# target encoder & ohe & LogisticRegression

In [1]:
%load_ext autoreload
%autoreload 2

import os
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(os.path.abspath('..'))
# ---------------------------------
import numpy as np
import pandas as pd
import scipy
import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.metrics import roc_auc_score

from hyperopt import hp
# ---------------------------------
from tools import CV, Tuning, CVGetScore, IdxValEncoder, linear_regression

In [2]:
train_df = pd.read_csv('../data/train.csv', index_col='id')
test_df = pd.read_csv('../data/test.csv', index_col='id')

# ord_5
for i in range(2):
    train_df[f'ord_5_{i}'] = train_df['ord_5'].str[i]
    test_df[f'ord_5_{i}'] = test_df['ord_5'].str[i]

# fillna
for col in test_df.columns:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)

# target
target = train_df['target']
y_train = target.values

# drop
train_df.drop(['target', 'ord_5'], axis=1, inplace=True)
test_df.drop(['ord_5'], axis=1, inplace=True)

In [3]:
feature_col = train_df.columns

bin_col = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']

class_col = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4',
             'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
             'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4',
             'day', 'month', 'ord_5_0', 'ord_5_1']

In [4]:
for col in bin_col:
    map_dict = dict(zip(train_df[col].unique(), [0., 1.]))
    train_df[col] = train_df[col].map(map_dict)
    test_df[col] = test_df[col].map(map_dict)

In [5]:
ecd = IdxValEncoder(feature_col, bin_col, class_col)
ecd.fit(train_df, verbose=1)
ecd.get_vocabulary()

600000it [00:11, 53190.93it/s]


5607

In [6]:
ecd.fit(test_df, verbose=1)
ecd.get_vocabulary()

400000it [00:07, 51319.70it/s]


5608

In [7]:
idx, val = ecd.transform(train_df, verbose=1)
idx_test, val_test = ecd.transform(test_df, verbose=1)

600000it [00:12, 47409.06it/s]
400000it [00:08, 47885.95it/s]


# fit once

In [13]:
learning_rate =1e-4
batch_size =  8192
epochs = 100

model_param = {'vocabulary_size':ecd.get_vocabulary(), 
               'feature_number': len(feature_col),
               'activation': 'sigmoid',
               'metrics': ['AUC'],
               'optimizer': tf.keras.optimizers.Adam(learning_rate=learning_rate),
               'loss': tf.keras.losses.BinaryCrossentropy(),
               'l1':0.,
               'l2': 0.}

callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=2, mode='max')
fit_param = {'batch_size': 8192, 'epochs':epochs, 'verbose': 1, 'callbacks':[callback]}

model = linear_regression(**model_param)

cv = CV(model, 5)

cv.fit(x=[idx, val],
       y=y_train, 
       metrics_func=roc_auc_score,
       split_method=StratifiedKFold,
       fit_params=fit_param,
       eval_param={'batch_size':batch_size},
       use_proba=False, 
       verbose=True,
       fit_use_valid=True)

Train on 480000 samples, validate on 120000 samples
Epoch 1/100
480000/480000 [==============================] - 2s 3us/sample - loss: 1.9388 - AUC: 0.4664 - val_loss: 1.7695 - val_AUC: 0.4638
Epoch 2/100
480000/480000 [==============================] - 1s 2us/sample - loss: 1.6223 - AUC: 0.4653 - val_loss: 1.4758 - val_AUC: 0.4633
Epoch 3/100
480000/480000 [==============================] - 1s 2us/sample - loss: 1.3502 - AUC: 0.4657 - val_loss: 1.2262 - val_AUC: 0.4643
Epoch 4/100
480000/480000 [==============================] - 1s 2us/sample - loss: 1.1212 - AUC: 0.4668 - val_loss: 1.0189 - val_AUC: 0.4669
Epoch 5/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.9348 - AUC: 0.4693 - val_loss: 0.8539 - val_AUC: 0.4698
Epoch 6/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.7895 - AUC: 0.4727 - val_loss: 0.7289 - val_AUC: 0.4739
Epoch 7/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.6840 - AUC: 0.476

480000/480000 [==============================] - 1s 2us/sample - loss: 0.5277 - AUC: 0.6159 - val_loss: 0.5277 - val_AUC: 0.6143
Epoch 59/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5274 - AUC: 0.6165 - val_loss: 0.5274 - val_AUC: 0.6149
Epoch 60/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5271 - AUC: 0.6170 - val_loss: 0.5271 - val_AUC: 0.6153
Epoch 61/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5268 - AUC: 0.6174 - val_loss: 0.5269 - val_AUC: 0.6158
Epoch 62/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5266 - AUC: 0.6178 - val_loss: 0.5266 - val_AUC: 0.6163
Epoch 63/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5263 - AUC: 0.6181 - val_loss: 0.5264 - val_AUC: 0.6167
Epoch 64/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5261 - AUC: 0.6184 - val_loss: 0.5261 - val_AUC: 0.6170
Epoch 65/100
480000

480000/480000 [==============================] - 1s 2us/sample - loss: 0.5260 - AUC: 0.6183 - val_loss: 0.5260 - val_AUC: 0.6171
Epoch 42/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5258 - AUC: 0.6187 - val_loss: 0.5258 - val_AUC: 0.6173
Epoch 43/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5255 - AUC: 0.6188 - val_loss: 0.5256 - val_AUC: 0.6174
Epoch 44/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5253 - AUC: 0.6189 - val_loss: 0.5254 - val_AUC: 0.6175
Epoch 45/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5251 - AUC: 0.6189 - val_loss: 0.5252 - val_AUC: 0.6176
Epoch 46/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5249 - AUC: 0.6190 - val_loss: 0.5250 - val_AUC: 0.6176
Epoch 47/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5247 - AUC: 0.6190 - val_loss: 0.5248 - val_AUC: 0.6176
folds 1 is done, sc

480000/480000 [==============================] - 1s 2us/sample - loss: 0.5698 - AUC: 0.5021 - val_loss: 0.5657 - val_AUC: 0.5114
Epoch 5/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5632 - AUC: 0.5125 - val_loss: 0.5602 - val_AUC: 0.5214
Epoch 6/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5585 - AUC: 0.5215 - val_loss: 0.5561 - val_AUC: 0.5299
Epoch 7/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5549 - AUC: 0.5299 - val_loss: 0.5529 - val_AUC: 0.5384
Epoch 8/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5519 - AUC: 0.5376 - val_loss: 0.5502 - val_AUC: 0.5459
Epoch 9/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5495 - AUC: 0.5445 - val_loss: 0.5480 - val_AUC: 0.5520
Epoch 10/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5474 - AUC: 0.5505 - val_loss: 0.5461 - val_AUC: 0.5574
Epoch 11/100
480000/4800

480000/480000 [==============================] - 1s 2us/sample - loss: 0.5402 - AUC: 0.5748 - val_loss: 0.5398 - val_AUC: 0.5742
Epoch 16/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5391 - AUC: 0.5783 - val_loss: 0.5388 - val_AUC: 0.5778
Epoch 17/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5381 - AUC: 0.5818 - val_loss: 0.5378 - val_AUC: 0.5814
Epoch 18/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5372 - AUC: 0.5852 - val_loss: 0.5369 - val_AUC: 0.5847
Epoch 19/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5364 - AUC: 0.5883 - val_loss: 0.5361 - val_AUC: 0.5877
Epoch 20/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5356 - AUC: 0.5913 - val_loss: 0.5353 - val_AUC: 0.5907
Epoch 21/100
480000/480000 [==============================] - 1s 2us/sample - loss: 0.5348 - AUC: 0.5940 - val_loss: 0.5346 - val_AUC: 0.5937
Epoch 22/100
480000

0.6190376107314561

# Tuning

In [ ]:
# fit param
callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=2, mode='max')
fit_param = {'batch_size': 8192, 'epochs':200, 'verbose': 0, 'callbacks':[callback]}

cv_fit_param = {'fit_params': fit_param, 
                'eval_param': {'batch_size':8192},
                'use_proba':False, 
                'fit_use_valid': True}

# model_fix_param & model_search_space
opt = tf.keras.optimizers.Adam(learning_rate=5e-4)

model_fix_param = {'vocabulary_size':ecd.get_vocabulary(), 
                   'feature_number': len(feature_col),
                   'activation': 'sigmoid',
                   'metrics': ['AUC'],
                   'optimizer': opt}

model_search_space = {'loss': hp.choice('loss', ['MSE', tf.keras.losses.BinaryCrossentropy()])}

# cv get score
def neg_auc(y_true, y_pred):
    return - roc_auc_score(y_true, y_pred)

gs = CVGetScore(x=[idx, val],
                y=y_train, 
                metrics_func=neg_auc,
                split_method=StratifiedKFold,
                nfolds=5, 
                random_state=2333,
                model=linear_regression, 
                cv_fit_params=cv_fit_param, 
                model_fix_params=model_fix_param, 
                model_search_space=model_search_space)

tuning = Tuning(gs, verbose=1)
tuning.fmin(gs.GET_SEARCH_SPACE(), max_evals=2)

# CV training use Low Learning Rate

In [ ]:
batch_size = 8192
epochs = 20
learning_rate = 1e-3
patience = 1

# fit param
callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=patience, mode='max')
fit_param = {'batch_size': batch_size, 'epochs':epochs, 'verbose': 1, 'callbacks':[callback]}

# model_fix_param & model_search_space
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model_param = {'vocabulary_size':ecd.get_vocabulary(), 
               'feature_number': len(feature_col),
               'activation': 'sigmoid',
               'metrics': ['AUC'],
               'optimizer': opt, 
               'loss': tf.keras.losses.BinaryCrossentropy(),
               'l1':0., 
               'l2':0.}

model = linear_regression(**model_param)
cv = CV(model, 10)

cv.fit(x=[idx, val],
       y=y_train, 
       metrics_func=roc_auc_score,
       split_method=StratifiedKFold,
       fit_params=fit_param,
       eval_param={'batch_size':batch_size},
       use_proba=False, 
       verbose=True,
       fit_use_valid=True)

In [ ]:
model = cv.model
opt = tf.keras.optimizers.Adam(learning_rate=2e-5)
for submodel in model:
    submodel.compile(loss=tf.keras.losses.BinaryCrossentropy(), metrics=['AUC'], optimizer=opt)

In [ ]:
batch_size = 8192
epochs = 400

# fit param
callback = tf.keras.callbacks.EarlyStopping(monitor='val_AUC',
                                            patience=5,
                                            mode='max')

fit_param = {
    'batch_size': batch_size,
    'epochs':epochs, 
    'verbose': 1, 
    'callbacks':[callback]
}

cv = CV(model, 10)

cv.fit(x=[idx, val],
       y=y_train, 
       metrics_func=roc_auc_score,
       split_method=StratifiedKFold,
       fit_params=fit_param,
       eval_param={'batch_size':batch_size},
       use_proba=False, 
       verbose=True,
       fit_use_valid=True)

# predict

In [ ]:
score = cv.predict(x=[idx_test, val_test], pred_param={'batch_size': batch_size})

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv', index_col='id')
submission['target'] = score
submission.to_csv('../tmp/submission/main_1_lr.csv')